In [105]:
import pandas as pd
import numpy as np
from copy import deepcopy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline

import time
import pickle


# import

In [2]:
df = pd.read_csv('processed_remsy\metacritic_comments_remsy.csv')
df

,Username,Userscore,Title,Comment
0,SirCaestus,10.0,The Legend of Zelda: Ocarina of Time,"Everything in OoT is so near at perfection, it..."
1,Kaistlin,10.0,The Legend of Zelda: Ocarina of Time,I won't bore you with what everyone is already...
2,Jacody,10.0,The Legend of Zelda: Ocarina of Time,Anyone who gives the masterpiece below a 7 or ...
3,doodlerman,10.0,The Legend of Zelda: Ocarina of Time,I'm one of those people who think that this is...
4,StevenA,10.0,The Legend of Zelda: Ocarina of Time,This game is the highest rated game on Metacr...
...,...,...,...,...
269962,RileyWRussell,7.0,Etrian Odyssey Untold: The Millennium Girl,"Extremely similar to EO:4, which obviously isn..."
269963,TemplarGR,NaN,Etrian Odyssey Untold: The Millennium Girl,Typical overrated Atlus trash. A game i should...
269964,midipon,9.0,Etrian Odyssey Untold: The Millennium Girl,While I find the story mode to have annoying c...
269965,night4,8.0,Etrian Odyssey Untold: The Millennium Girl,"Pretty good, but it certainly lacks the visual..."


In [60]:
df1 = df[df['Userscore'].notna() == True]

In [61]:
df1.shape

(250825, 4)

In [58]:
user_rating_matrix = df.pivot_table(index=['Username'], columns=['Title'], values=['Userscore'], aggfunc='mean')
user_rating_matrix.head()

Userscore                               \
Title                    'Splosion Man 007: The World is Not Enough   
Username                                                              
"Harv'sNoBroofMine"Smith           NaN                          NaN   
$ERIC$                             NaN                          NaN   
$GMoney$GIrdham                    NaN                          NaN   
$MOB$-DevilDoggSince96             NaN                          NaN   
'Airstrike'I                       NaN                          NaN   

                                                                              \
Title                    2010 FIFA World Cup South Africa 3D After Burner II   
Username                                                                       
"Harv'sNoBroofMine"Smith                              NaN                NaN   
$ERIC$                                                NaN                NaN   
$GMoney$GIrdham                                       NaN                NaN   
$MOB$-DevilDoggSince96                                NaN                NaN   
'Airstrike'I                                          NaN                NaN   

                                                                               \
Title                    3D Gunstar Heroes 3D Out Run 3D Sonic the Hedgehog 2   
Username                                                                        
"Harv'sNoBroofMine"Smith               NaN        NaN                     NaN   
$ERIC$                                 NaN        NaN                     NaN   
$GMoney$GIrdham                        NaN        NaN                     NaN   
$MOB$-DevilDoggSince96                 NaN        NaN                     NaN   
'Airstrike'I                           NaN        NaN                     NaN   

                                                                    ...  \
Title                    3D Streets of Rage II 80 Days (2015) ABZU  ...   
Username                                                            ...   
"Harv'sNoBroofMine"Smith                   NaN            NaN  NaN  ...   
$ERIC$                                     NaN            NaN  NaN  ...   
$GMoney$GIrdham                            NaN            NaN  NaN  ...   
$MOB$-DevilDoggSince96                     NaN            NaN  NaN  ...   
'Airstrike'I                               NaN            NaN  NaN  ...   

                                                                      \
Title                    Zeus: Master of Olympus Zombie Night Terror   
Username                                                               
"Harv'sNoBroofMine"Smith                     NaN                 NaN   
$ERIC$                                       NaN                 NaN   
$GMoney$GIrdham                              NaN                 NaN   
$MOB$-DevilDoggSince96                       NaN                 NaN   
'Airstrike'I                                 NaN                 NaN   

                                                                     \
Title                    Zone of the Enders: The 2nd Runner de Blob   
Username                                                              
"Harv'sNoBroofMine"Smith                                NaN     NaN   
$ERIC$                                                  NaN     NaN   
$GMoney$GIrdham                                         NaN     NaN   
$MOB$-DevilDoggSince96                                  NaN     NaN   
'Airstrike'I                                            NaN     NaN   

                                                                            \
Title                    echochrome escapeVektor: Chapter 1 flower ilomilo   
Username                                                                     
"Harv'sNoBroofMine"Smith        NaN                     NaN    NaN     NaN   
$ERIC$                          NaN                     NaN    NaN     NaN   
$GMoney$GIrdham                 N

In [59]:
user_rating_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 117810 entries, "Harv'sNoBroofMine"Smith to ÜbermenschR
Columns: 2323 entries, ('Userscore', "'Splosion Man") to ('Userscore', 'inFamous 2')
dtypes: float64(2323)
memory usage: 2.0+ GB


# Getting the data ready for Surprise

In [62]:
data = df1[['Username', 'Title', 'Userscore']]

reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_df(data, reader=reader)

In [63]:
# train test split

trainset, testset = train_test_split(data, test_size=.2, random_state = 2333)

In [64]:
print('Type trainset :',type(trainset),'\n')
print('Type testset :',type(testset))

Type trainset : <class 'surprise.trainset.Trainset'> 

Type testset : <class 'list'>


In [65]:
print(len(testset))
print(testset[0])

50165
('Naessarne', 'Divinity: Original Sin II', 8.0)


In [50]:
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items, '\n')

Number of users:  109730 

Number of items:  2309 



KNN

- baseline

In [66]:
sim_cos = {'name':'cosine', 'user_based':False}

In [67]:
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [68]:
basic.sim

array([[1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [69]:
predictions = basic.test(testset)

In [70]:
print(accuracy.rmse(predictions))

RMSE: 4.0359
4.035938571338445


# SVD

- baseline

In [111]:
svd = SVD()
svd.fit(trainset)

predictions = svd.test(testset)
accuracy.rmse(predictions)

RMSE: 3.9951


3.995106898826457

- GridSearch

In [107]:
start = time.time()

params = {'n_factors': [20, 50, 100], 'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD, param_grid=params, n_jobs=-1, cv=5)
g_s_svd.fit(data)

end = time.time()
print(f'time spent {end-start}')

time spent 20.846306324005127


In [108]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

# slightly better

{'rmse': 3.9880269156619725, 'mae': 3.7042668679363695}
{'rmse': {'n_factors': 100, 'reg_all': 0.1}, 'mae': {'n_factors': 100, 'reg_all': 0.02}}


- predict

In [112]:
from collections import defaultdict

# given prediction for a set of users, get the top n ranked for each user 

def get_top_n(predictions, n=5):

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [113]:
predset = trainset.build_anti_testset() # builds a set of user-item interactions that have no observed rating
predictions = svd.test(predset) # predicts on this set

# order and rank top 5 items for users that they have not interacted with
top_n = get_top_n(predictions, n=5) 

KeyboardInterrupt: 